In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from selenium.webdriver.common.by import By
from textblob import TextBlob
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
import quandl
import requests
import time

import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
from tensorboard import notebook
from tensorflow.contrib.tensor_forest.python import tensor_forest
import ast
from sklearn import preprocessing
import os


In [2]:
tf.enable_eager_execution()

In [3]:
%load_ext tensorboard

In [4]:
csv = pd.read_csv("C:/Users/Adhithya/Desktop/sector_output.csv")

In [4]:
def train_model(input_neurons, hidden_layers, output_neurons, opt, dropout):
    print(input_neurons)
    nn_model = tf.keras.models.Sequential()
    nn_model.add(keras.layers.InputLayer(input_shape=(1,input_neurons)))
    for neurons in hidden_layers:
        print(neurons)
        nn_model.add(tf.keras.layers.Dense(neurons, activation=tf.nn.relu))
    print(output_neurons)
    nn_model.add(tf.keras.layers.Dropout(dropout))
    nn_model.add(keras.layers.Dense(output_neurons, activation=tf.nn.softmax))
    
    nn_model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    return nn_model

In [5]:
def populate_training_testing(train_data, test_data):
    
    testing_inputs = []
    testing_outputs = []
    training_inputs = []
    training_outputs = []
    
    for i,row in test_data.iterrows():
        features = populate_features(row)
#         this_test = [row['token']]
#         this_test.extend([features, row['labels'], row['bounding box']])
#         test_array.append(this_test)
        
#         this_output = ast.literal_eval(row['labels'])
#         this_output = ast.literal_eval(row['labels'])
        target = row['Prev Target']
        testing_outputs.append(target)

        testing_inputs.append(features)
    
    for i,row in train_data.iterrows():
        features = populate_features(row)
#         this_output = ast.literal_eval(row['labels'])
        target = row['Prev Target']
        training_outputs.append(target)

        training_inputs.append(features)
    
    return training_inputs, training_outputs, testing_inputs, testing_outputs

In [140]:
def scale_average(starting, average):
    starting = float(starting)
    average = float(average)
    perc_change = (average - starting)/average
    return perc_change

csv["Scaled 200"] = list(map(scale_average, csv["200 Day Moving Average"], csv["Starting"]))
csv["Scaled 90"] = list(map(scale_average, csv["90 Day Moving Average"], csv["Starting"]))
csv["Scaled 30"] = list(map(scale_average, csv["30 Day Moving Average"], csv["Starting"]))

In [141]:
def target_func(end, start):
    if float(end) > float(start):
        return 1
    else:
        return 0
    

csv["target"] = list(map(target_func, csv["Closing"], csv["Starting"]))

In [142]:
volumes_list = []
for symbol in csv["Symbol"].unique():
    volumes = preprocessing.scale(csv[csv["Symbol"] == symbol]["Volume"].values)
    volumes_list.extend(volumes)

csv["Scaled Volumes"] = volumes_list
cleaned_csv = csv.dropna(axis=0)

In [6]:
def populate_features(row):
    features = []
    volume = row['Scaled Volumes']
    snp = row['snp_change']
    nyse = row['nyse_change']
    nasdaq = row['nasdaq_change']
    two_hundred = row['Scaled 200']
    ninety = row['Scaled 90']
    thirty = row['Scaled 30']
    sector = row['Sector Change']
#     features.extend([volume, snp, nyse, nasdaq, two_hundred, ninety, thirty, sector])
    features.extend([volume, two_hundred, ninety, thirty, snp, nyse, nasdaq])
    return features 

In [7]:
def reshape(input_data):
    return np.array(input_data).reshape(len(input_data),1, len(input_data[0]))

In [146]:
cleaned_csv["Prev Target"] = cleaned_csv['target'].shift(-1)

C:\Users\Adhithya\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [147]:
cleaned_csv

,Unnamed: 0,Symbol,Date,Starting,High,Low,Closing,Volume,Week,snp_change,...,90 Day Moving Average,30 Day Moving Average,Sector,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target
143,143,A,2015-05-11,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,...,43.341449,46.990345,Healthcare,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0
144,144,A,2015-05-12,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,...,43.331159,46.965862,Healthcare,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0
145,145,A,2015-05-13,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,...,43.329710,46.962414,Healthcare,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0
146,146,A,2015-05-14,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,...,43.333188,46.970690,Healthcare,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0
148,148,A,2015-05-18,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,...,43.341594,46.825667,Healthcare,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0
149,149,A,2015-05-19,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,...,43.337826,46.817000,Healthcare,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0
150,150,A,2015-05-20,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,...,43.341304,46.825000,Healthcare,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0
151,151,A,2015-05-21,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,...,43.337101,46.815333,Healthcare,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0
152,152,A,2015-05-22,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,...,43.339710,46.821333,Healthcare,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0
154,154,A,2015-05-26,42.48,42.6300,41.980,42.06,1919600,201522,-0.009947,...,43.333333,46.806667,Healthcare,-0.006935,0.006546,-0.020088,-0.101852,0,-0.371681,1.0


In [148]:
cleaned_csv.drop(cleaned_csv.columns[0], axis=1)

,Symbol,Date,Starting,High,Low,Closing,Volume,Week,snp_change,nyse_change,...,90 Day Moving Average,30 Day Moving Average,Sector,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target
143,A,2015-05-11,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,-0.003701,...,43.341449,46.990345,Healthcare,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0
144,A,2015-05-12,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,0.000637,...,43.331159,46.965862,Healthcare,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0
145,A,2015-05-13,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,-0.001305,...,43.329710,46.962414,Healthcare,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0
146,A,2015-05-14,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,0.003886,...,43.333188,46.970690,Healthcare,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0
148,A,2015-05-18,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,0.001862,...,43.341594,46.825667,Healthcare,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0
149,A,2015-05-19,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,-0.000114,...,43.337826,46.817000,Healthcare,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0
150,A,2015-05-20,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,-0.000995,...,43.341304,46.825000,Healthcare,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0
151,A,2015-05-21,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,0.002421,...,43.337101,46.815333,Healthcare,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0
152,A,2015-05-22,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,-0.001449,...,43.339710,46.821333,Healthcare,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0
154,A,2015-05-26,42.48,42.6300,41.980,42.06,1919600,201522,-0.009947,-0.011813,...,43.333333,46.806667,Healthcare,-0.006935,0.006546,-0.020088,-0.101852,0,-0.371681,1.0


In [152]:
cleaned_csv = cleaned_csv.set_index("Date")

In [154]:
cleaned_csv

,Unnamed: 0,Symbol,Starting,High,Low,Closing,Volume,Week,snp_change,nyse_change,...,90 Day Moving Average,30 Day Moving Average,Sector,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target
Date,,,,,,,,,,,,,,,,,,,,,
2015-05-11,143,A,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,-0.003701,...,43.341449,46.990345,Healthcare,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0
2015-05-12,144,A,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,0.000637,...,43.331159,46.965862,Healthcare,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0
2015-05-13,145,A,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,-0.001305,...,43.329710,46.962414,Healthcare,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0
2015-05-14,146,A,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,0.003886,...,43.333188,46.970690,Healthcare,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0
2015-05-18,148,A,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,0.001862,...,43.341594,46.825667,Healthcare,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0
2015-05-19,149,A,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,-0.000114,...,43.337826,46.817000,Healthcare,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0
2015-05-20,150,A,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,-0.000995,...,43.341304,46.825000,Healthcare,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0
2015-05-21,151,A,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,0.002421,...,43.337101,46.815333,Healthcare,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0
2015-05-22,152,A,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,-0.001449,...,43.339710,46.821333,Healthcare,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0


In [8]:
cleaned_csv = pd.read_csv("C:/Users/Adhithya/Desktop/cleaned.csv")

In [9]:
cleaned_csv = cleaned_csv.dropna()

In [10]:
train_data = cleaned_csv[cleaned_csv['Symbol'] != 'A']
test_data = cleaned_csv[cleaned_csv['Symbol'] == 'A']

In [11]:
training_inputs, training_outputs, testing_inputs, testing_outputs  = populate_training_testing(train_data, test_data)

In [12]:
training_inputs, testing_inputs = reshape(training_inputs), reshape(testing_inputs)
model_data = training_inputs,  testing_inputs, training_outputs, testing_outputs

In [13]:
cleaned_csv

,Date,Unnamed: 0,Symbol,Starting,High,Low,Closing,Volume,Week,snp_change,...,90 Day Moving Average,30 Day Moving Average,Sector,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target
0,2015-05-11,143,A,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,...,43.341449,46.990345,Healthcare,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0
1,2015-05-12,144,A,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,...,43.331159,46.965862,Healthcare,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0
2,2015-05-13,145,A,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,...,43.329710,46.962414,Healthcare,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0
3,2015-05-14,146,A,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,...,43.333188,46.970690,Healthcare,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0
4,2015-05-18,148,A,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,...,43.341594,46.825667,Healthcare,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0
5,2015-05-19,149,A,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,...,43.337826,46.817000,Healthcare,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0
6,2015-05-20,150,A,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,...,43.341304,46.825000,Healthcare,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0
7,2015-05-21,151,A,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,...,43.337101,46.815333,Healthcare,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0
8,2015-05-22,152,A,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,...,43.339710,46.821333,Healthcare,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0
9,2015-05-26,154,A,42.48,42.6300,41.980,42.06,1919600,201522,-0.009947,...,43.333333,46.806667,Healthcare,-0.006935,0.006546,-0.020088,-0.101852,0,-0.371681,1.0


In [21]:
testing_inputs.shape

(270, 1, 7)

In [55]:
nn_model = train_model(7, [10, 6], 2, "adam", 0.2)

7
10
6
2


In [56]:
nn_model.fit(training_inputs, np.array(training_outputs), epochs=5)
test_loss, test_acc = nn_model.evaluate(testing_inputs, testing_outputs)
print('Test accuracy:', test_acc)

Epoch 1/5
288433/288433 [==============================] - 17s 59us/sample - loss: 0.6931 - acc: 0.5094
Epoch 2/5
288433/288433 [==============================] - 17s 58us/sample - loss: 0.6930 - acc: 0.5100
Epoch 3/5
288433/288433 [==============================] - 17s 59us/sample - loss: 0.6930 - acc: 0.5103
Epoch 4/5
288433/288433 [==============================] - 16s 57us/sample - loss: 0.6930 - acc: 0.5104
Epoch 5/5
270/270 [==============================] - 0s 1ms/sample - loss: 0.6918 - acc: 0.5284
Test accuracy: 0.5283739


In [103]:
predictions = nn_model.predict(testing_inputs)

In [48]:
predictions

array([[[0.39287588, 0.6071241 ]],

       [[0.5029774 , 0.4970227 ]],

       [[0.5539958 , 0.4460042 ]],

       [[0.2575083 , 0.7424918 ]],

       [[0.22941078, 0.77058923]],

       [[0.5453895 , 0.45461056]],

       [[0.40323377, 0.59676623]],

       [[0.5544524 , 0.44554755]],

       [[0.33468476, 0.6653152 ]],

       [[0.7776768 , 0.22232313]],

       [[0.19776514, 0.8022348 ]],

       [[0.48313162, 0.5168684 ]],

       [[0.5508936 , 0.44910643]],

       [[0.51189697, 0.48810306]],

       [[0.41227055, 0.5877295 ]],

       [[0.36416027, 0.6358397 ]],

       [[0.6248567 , 0.37514332]],

       [[0.17957844, 0.8204216 ]],

       [[0.71784765, 0.28215232]],

       [[0.69782054, 0.30217952]],

       [[0.21549906, 0.784501  ]],

       [[0.34414396, 0.655856  ]],

       [[0.63769096, 0.36230898]],

       [[0.35333398, 0.64666605]],

       [[0.2641005 , 0.7358995 ]],

       [[0.4035857 , 0.5964142 ]],

       [[0.15099798, 0.84900194]],

       [[0.5940905 , 0.40590

In [14]:
cleaned_csv['3 Month Future'] =  cleaned_csv['Closing'].shift(-63)
cleaned_csv['1 Month Future'] =  cleaned_csv['Closing'].shift(-21)
cleaned_csv['1 Week Future'] =  cleaned_csv['Closing'].shift(-4)

In [15]:
cleaned_csv

,Date,Unnamed: 0,Symbol,Starting,High,Low,Closing,Volume,Week,snp_change,...,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target,3 Month Future,1 Month Future,1 Week Future
0,2015-05-11,143,A,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,...,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0,38.89,40.53,42.63
1,2015-05-12,144,A,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,...,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0,38.56,39.84,42.37
2,2015-05-13,145,A,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,...,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0,38.65,39.52,42.61
3,2015-05-14,146,A,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,...,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0,38.82,39.79,42.32
4,2015-05-18,148,A,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,...,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0,39.02,39.60,42.50
5,2015-05-19,149,A,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,...,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0,38.40,39.90,42.06
6,2015-05-20,150,A,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,...,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0,37.51,39.49,42.61
7,2015-05-21,151,A,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,...,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0,36.23,39.81,41.75
8,2015-05-22,152,A,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,...,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0,34.68,39.60,41.19
9,2015-05-26,154,A,42.48,42.6300,41.980,42.06,1919600,201522,-0.009947,...,-0.006935,0.006546,-0.020088,-0.101852,0,-0.371681,1.0,34.36,40.19,40.92


In [16]:
def improvement_perc(average, start, future):
    start = float(start)
    future = float(future)
    if (average > 0 and future > start) or (average < 0 and future < start):
        return 1
    else:
        return 0
    
week_improvement_200 = list(map(improvement_perc, cleaned_csv["Scaled 200"], cleaned_csv["Starting"], cleaned_csv['1 Week Future']))
month_improvement_200 = list(map(improvement_perc, cleaned_csv["Scaled 200"], cleaned_csv["Starting"], cleaned_csv['1 Month Future']))
month3_improvement_200 = list(map(improvement_perc, cleaned_csv["Scaled 200"], cleaned_csv["Starting"], cleaned_csv['3 Month Future']))

week_improvement_90 = list(map(improvement_perc, cleaned_csv["Scaled 90"], cleaned_csv["Starting"], cleaned_csv['1 Week Future']))
month_improvement_90 = list(map(improvement_perc, cleaned_csv["Scaled 90"], cleaned_csv["Starting"], cleaned_csv['1 Month Future']))
month3_improvement_90 = list(map(improvement_perc, cleaned_csv["Scaled 90"], cleaned_csv["Starting"], cleaned_csv['3 Month Future']))

week_improvement_30 = list(map(improvement_perc, cleaned_csv["Scaled 30"], cleaned_csv["Starting"], cleaned_csv['1 Week Future']))
month_improvement_30 = list(map(improvement_perc, cleaned_csv["Scaled 30"], cleaned_csv["Starting"], cleaned_csv['1 Month Future']))
month3_improvement_30 = list(map(improvement_perc, cleaned_csv["Scaled 30"], cleaned_csv["Starting"], cleaned_csv['3 Month Future']))

In [17]:
week_perc_200 = sum(week_improvement_200) / len(cleaned_csv)
month_perc_200 = sum(month_improvement_200) / len(cleaned_csv)
month3_perc_200 = sum(month3_improvement_200) / len(cleaned_csv)

week_perc_90 = sum(week_improvement_90) / len(cleaned_csv)
month_perc_90 = sum(month_improvement_90) / len(cleaned_csv)
month3_perc_90 = sum(month3_improvement_90) / len(cleaned_csv)

week_perc_30 = sum(week_improvement_30) / len(cleaned_csv)
month_perc_30 = sum(month_improvement_30) / len(cleaned_csv)
month3_perc_30 = sum(month3_improvement_30) / len(cleaned_csv)

In [18]:
print(week_perc_200, week_perc_90, week_perc_30)

0.5042829482201432 0.503576339698583 0.5027484993228335


In [19]:
print(month_perc_200, month_perc_90, month_perc_30)

0.4977676019992865 0.49819364537258015 0.49734155862599283


In [20]:
print(month3_perc_200, month3_perc_90, month3_perc_30)

0.4748547815575176 0.47555792631181526 0.4750695351277957


In [22]:
count = 0
for i, row in cleaned_csv.iterrows():
    if (row["Sector Change"] < 0 and row['target'] == 0) or (row["Sector Change"] > 0 and row['target'] == 1):
         count += 1
    
count/len(cleaned_csv)

0.6901521632958438

In [23]:
cleaned_csv

,Date,Unnamed: 0,Symbol,Starting,High,Low,Closing,Volume,Week,snp_change,...,Sector Change,Scaled 200,Scaled 90,Scaled 30,target,Scaled Volumes,Prev Target,3 Month Future,1 Month Future,1 Week Future
0,2015-05-11,143,A,42.39,42.8300,42.390,42.62,975400,201520,-0.004836,...,0.004799,0.011232,-0.022445,-0.108524,1,-1.123086,0.0,38.89,40.53,42.63
1,2015-05-12,144,A,42.36,42.5000,41.500,41.91,2617800,201520,-0.001783,...,-0.002031,0.003729,-0.022926,-0.108731,0,0.183954,0.0,38.56,39.84,42.37
2,2015-05-13,145,A,42.05,42.3900,41.710,41.81,1992700,201520,-0.000543,...,-0.001286,-0.003599,-0.030433,-0.116823,0,-0.313508,0.0,38.65,39.52,42.61
3,2015-05-14,146,A,42.18,42.2300,41.870,42.05,2883600,201520,0.009841,...,0.009137,-0.000545,-0.027340,-0.113577,0,0.395481,1.0,38.82,39.79,42.32
4,2015-05-18,148,A,42.05,42.7000,41.980,42.63,1966100,201521,0.003724,...,0.010456,-0.003708,-0.030716,-0.113571,1,-0.334676,1.0,39.02,39.60,42.50
5,2015-05-19,149,A,41.57,42.8300,41.500,42.37,5277300,201521,-0.000761,...,-0.001467,-0.015254,-0.042526,-0.126221,1,2.300414,1.0,38.40,39.90,42.06
6,2015-05-20,150,A,42.50,42.8400,42.430,42.61,2652300,201521,-0.000912,...,0.004183,0.006923,-0.019795,-0.101765,1,0.211409,0.0,37.51,39.49,42.61
7,2015-05-21,151,A,42.57,42.7300,42.200,42.32,2126200,201521,0.002479,...,-0.004966,0.008603,-0.018020,-0.099726,0,-0.207267,1.0,36.23,39.81,41.75
8,2015-05-22,152,A,42.27,42.6700,42.070,42.50,1484700,201521,-0.002018,...,0.007152,0.001538,-0.025307,-0.107673,1,-0.717780,0.0,34.68,39.60,41.19
9,2015-05-26,154,A,42.48,42.6300,41.980,42.06,1919600,201522,-0.009947,...,-0.006935,0.006546,-0.020088,-0.101852,0,-0.371681,1.0,34.36,40.19,40.92


In [135]:
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([8,12,16]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([4,6,8]))
# HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1,2,3]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.contrib.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS_1, HP_NUM_UNITS_2, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [141]:
def train_test_model(hparams, data):
    training_inputs, testing_inputs, training_outputs, testing_outputs = data[0], data[1], data[2], data[3]
    nn_model = tf.keras.models.Sequential()
    nn_model.add(keras.layers.InputLayer(input_shape=(1,training_inputs.shape[2])))
#     for x in range(hparams[HP_NUM_LAYERS]):
#         nn_model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu))
    nn_model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS_1], activation=tf.nn.relu))
    nn_model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))
    nn_model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS_2], activation=tf.nn.relu))
    nn_model.add(keras.layers.Dense(2, activation=tf.nn.softmax))

    nn_model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
    )

    nn_model.fit(training_inputs, training_outputs, epochs=5)
    _, accuracy = nn_model.evaluate(testing_inputs, testing_outputs)
    return accuracy

In [142]:
def run(run_dir, hparams, model_data):
    with tf.contrib.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams) 
        accuracy = train_test_model(hparams, model_data)
        tf.summary.scalar(METRIC_ACCURACY, accuracy)

In [143]:
def run_hyperparameter_tuning(model_data):
    session_num = 0
    for num_units_1 in HP_NUM_UNITS_1.domain.values:
        for num_units_2 in HP_NUM_UNITS_2.domain.values:
            for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
                for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
    #                   HP_NUM_LAYERS: num_layers,
                      HP_NUM_UNITS_1: num_units_1,
                      HP_NUM_UNITS_2: num_units_2,
                      HP_DROPOUT: dropout_rate,
                      HP_OPTIMIZER: optimizer,
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run("logs/hparam_tuning/" + run_name, hparams, model_data)
                    session_num += 1

In [144]:
run_hyperparameter_tuning(model_data)

--- Starting trial: run-0
{'num_units_1': 8, 'num_units_2': 4, 'dropout': 0.1, 'optimizer': 'adam'}
./NeuralLogs\Layer_1:8Layer_2:4Optimizer:adam


W0823 12:29:36.823054 20104 deprecation.py:323] From C:\Users\Adhithya\Anaconda\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
288433/288433 [==============================] - 22s 77us/sample - loss: 0.6937 - acc: 0.5080
Epoch 2/5
288433/288433 [==============================] - 17s 59us/sample - loss: 0.6930 - acc: 0.5096
Epoch 3/5
288433/288433 [==============================] - 17s 58us/sample - loss: 0.6930 - acc: 0.5102
Epoch 4/5
288433/288433 [==============================] - 17s 58us/sample - loss: 0.6930 - acc: 0.5102
Epoch 5/5
270/270 [==============================] - 0s 858us/sample - loss: 0.6919 - acc: 0.5284
--- Starting trial: run-1
{'num_units_1': 8, 'num_units_2': 4, 'dropout': 0.1, 'optimizer': 'sgd'}
./NeuralLogs\Layer_1:8Layer_2:4Optimizer:sgd
Epoch 1/5
288433/288433 [==============================] - 16s 56us/sample - loss: 0.6932 - acc: 0.5084
Epoch 2/5
288433/288433 [==============================] - 16s 56us/sample - loss: 0.6931 - acc: 0.5094
Epoch 3/5
288433/288433 [==============================] - 16s 55us/sample - loss: 0.6930 - acc: 0.5098
Epoch 4/5
288433/288433 [=====

./NeuralLogs\Layer_1:8Layer_2:8Optimizer:sgd
Epoch 1/5
288433/288433 [==============================] - 17s 58us/sample - loss: 0.6940 - acc: 0.5064
Epoch 2/5
288433/288433 [==============================] - 16s 56us/sample - loss: 0.6931 - acc: 0.5083
Epoch 3/5
288433/288433 [==============================] - 16s 55us/sample - loss: 0.6931 - acc: 0.5084
Epoch 4/5
288433/288433 [==============================] - 16s 56us/sample - loss: 0.6931 - acc: 0.5091
Epoch 5/5
270/270 [==============================] - 0s 899us/sample - loss: 0.6925 - acc: 0.5284
--- Starting trial: run-12
{'num_units_1': 12, 'num_units_2': 4, 'dropout': 0.1, 'optimizer': 'adam'}
./NeuralLogs\Layer_1:12Layer_2:4Optimizer:adam
Epoch 1/5
288433/288433 [==============================] - 18s 63us/sample - loss: 0.6931 - acc: 0.5090
Epoch 2/5
288433/288433 [==============================] - 17s 60us/sample - loss: 0.6930 - acc: 0.5102
Epoch 3/5
288433/288433 [==============================] - 17s 59us/sample - loss: 0

270/270 [==============================] - 0s 844us/sample - loss: 0.6922 - acc: 0.5284
--- Starting trial: run-22
{'num_units_1': 12, 'num_units_2': 8, 'dropout': 0.2, 'optimizer': 'adam'}
./NeuralLogs\Layer_1:12Layer_2:8Optimizer:adam
Epoch 1/5
288433/288433 [==============================] - 18s 63us/sample - loss: 0.6936 - acc: 0.5078
Epoch 2/5
288433/288433 [==============================] - 18s 62us/sample - loss: 0.6930 - acc: 0.5090
Epoch 3/5
288433/288433 [==============================] - 17s 60us/sample - loss: 0.6930 - acc: 0.5097
Epoch 4/5
288433/288433 [==============================] - 19s 66us/sample - loss: 0.6929 - acc: 0.5094
Epoch 5/5
270/270 [==============================] - 0s 1ms/sample - loss: 0.6915 - acc: 0.4948
--- Starting trial: run-23
{'num_units_1': 12, 'num_units_2': 8, 'dropout': 0.2, 'optimizer': 'sgd'}
./NeuralLogs\Layer_1:12Layer_2:8Optimizer:sgd
Epoch 1/5
288433/288433 [==============================] - 18s 61us/sample - loss: 0.6936 - acc: 0.5074


288433/288433 [==============================] - 19s 65us/sample - loss: 0.6930 - acc: 0.5095
Epoch 5/5
270/270 [==============================] - 0s 921us/sample - loss: 0.6919 - acc: 0.5284
--- Starting trial: run-33
{'num_units_1': 16, 'num_units_2': 8, 'dropout': 0.1, 'optimizer': 'sgd'}
./NeuralLogs\Layer_1:16Layer_2:8Optimizer:sgd
Epoch 1/5
288433/288433 [==============================] - 18s 64us/sample - loss: 0.6937 - acc: 0.5064
Epoch 2/5
288433/288433 [==============================] - 18s 61us/sample - loss: 0.6932 - acc: 0.5080
Epoch 3/5
288433/288433 [==============================] - 18s 63us/sample - loss: 0.6931 - acc: 0.5084
Epoch 4/5
288433/288433 [==============================] - 18s 63us/sample - loss: 0.6930 - acc: 0.5085
Epoch 5/5
270/270 [==============================] - 0s 915us/sample - loss: 0.6922 - acc: 0.5284
--- Starting trial: run-34
{'num_units_1': 16, 'num_units_2': 8, 'dropout': 0.2, 'optimizer': 'adam'}
./NeuralLogs\Layer_1:16Layer_2:8Optimizer:ada